In [1]:
import pandas as pd
import requests 
import matplotlib.pyplot as plt
import os
from dotenv import load_dotenv

In [2]:
load_dotenv()

True

Задаём переменные окружения

In [3]:
DATE_BEGIN = os.getenv('DATE_BEGIN')
DATE_END = os.getenv('DATE_END')
API_URL = os.getenv('API_URL')

Зададим переменные для запросов

In [4]:
dates = {'begin': DATE_BEGIN, 'end': DATE_END}

Запросим данные по визитам, преобразуем их из json, преобразуем в df

In [ ]:
visits_request = requests.get(f'{API_URL}/visits', params=dates)
visits_data = visits_request.json()
visits_df = pd.DataFrame(visits_data)

Запросим данные по регистрациям, преобразуем их из json, преобразуем в df

In [ ]:
registrations_request = requests.get(f'{API_URL}/registrations', params=dates)
registrations_data = registrations_request.json()
registrations_df = pd.DataFrame(registrations_data)

Удалим боты из визитов

In [ ]:
visits_df = visits_df[~visits_df['user_agent'].str.contains('bot', case=False, na=False)]

Отбросим все визиты кроме последнего, возьмём только нужные колонки

In [ ]:
visits_sorted = visits_df.sort_values('datetime') 
visits_last = visits_sorted.drop_duplicates(subset="visit_id", keep="last")

Приведём datetime к дате, отбросим лишнее

In [ ]:
visits_last = visits_last.copy()
visits_last.loc[:, 'date_group'] = pd.to_datetime(visits_last['datetime']).dt.date

Отбросим лишние колонки, агрегируем данные по визитам, дадим правильное имя колонке с визитами

In [ ]:
grouped_unique_visits = visits_last.drop(['user_agent', 'datetime'], axis=1).groupby(['date_group', 'platform']).agg(visits=("visit_id", "count"))

Создадим колонку в датами в регистрациях, отбросим лишнее, группируем

In [ ]:
registrations_df['date_group'] = pd.to_datetime(registrations_df['datetime']).dt.date
grouped_registrations = registrations_df.drop(['registration_type', 'email', 'datetime'], axis=1).groupby(['date_group', 'platform']).agg(registrations=("user_id", "count"))

Смёржим регистрации с визитами по датам и платформам

In [ ]:
conversions = pd.merge(grouped_unique_visits, grouped_registrations, on=['date_group', 'platform'], how='left')
conversions['conversion'] = (conversions['registrations'] / conversions['visits'] * 100 )
conversions = conversions.reset_index()

Отсортируем по датам и создадим json

In [ ]:
conversions = conversions.sort_values('date_group')
conversions.to_json('conversion.json')

Прочитаем данные по рекламам и создадим колонку с датами, отбросим ненужные колонки

In [ ]:
ads_df = pd.read_csv('ads.csv')
ads_df = ads_df.sort_values('date') 

In [ ]:
ads_df.loc[:, 'date_group'] = pd.to_datetime(ads_df['date']).dt.date
ads_df = ads_df.drop(['date', 'utm_source', 'utm_medium'], axis=1)

Подготовим конверсии - сгруппируем по датам, отбросим лишнее

In [ ]:
conversions_dropped = conversions.drop(['platform', 'conversion'], axis=1)
conversions_dropped = conversions_dropped.groupby('date_group').agg('sum')

Смержим конверсии с рекламами

In [ ]:
conv_ads = pd.merge(conversions_dropped, ads_df,  on=['date_group'], how='left')
conv_ads = conv_ads[['date_group', 'visits', 'registrations', 'cost', 'utm_campaign']]

Заполним пропуски

In [ ]:
conv_ads['cost'] = conv_ads['cost'].fillna(0)
conv_ads['utm_campaign'] = conv_ads['utm_campaign'].fillna('none')

In [ ]:
conv_ads = conv_ads.sort_values('date_group')
conv_ads.to_json('ads.json')

Создаём папку для графиков

In [ ]:
os.mkdir('charts')

1. Итоговые визиты

In [ ]:
x = conv_ads['date_group']
y = conv_ads['visits']
plt.figure(figsize=(20, 5))
bars = plt.bar(x, y, color='#e66a56', label='Итоговые визиты')
for bar in bars:
    height = bar.get_height()
    plt.text(bar.get_x() + bar.get_width() / 2, height, f'{height:.0f}', ha='center', va='bottom')
#plt.legend()
plt.grid(axis='y', linewidth=0.5, alpha=0.5)
plt.xlabel('Дата')
plt.xlim([conv_ads['date_group'].min() - pd.Timedelta(days=2), conv_ads['date_group'].max() + pd.Timedelta(days=2)])
plt.xticks(x[::3], x[::3], rotation=45, fontsize=10)
plt.ylabel('Суммарное количество посещений сайта')
plt.title('Количество посещений сайта в отчётном периоде')
plt.tight_layout()
plt.savefig('./charts/visits.png')

2. Итоговые визиты с разбивкой по платформам: web, android, ios

In [ ]:
grouped_unique_visits = grouped_unique_visits.reset_index()

In [ ]:
pivot_visits = grouped_unique_visits.pivot(index='date_group', columns='platform', values='visits').fillna(0)
x = pivot_visits.index
plt.figure(figsize=(20, 10))
plt.bar(x, pivot_visits['web'], color='#acb78e', label='Web')
plt.bar(x, pivot_visits['android'], bottom=pivot_visits['web'], color='#aae11e', label='Android')
plt.bar(x, pivot_visits['ios'], bottom=pivot_visits['web'] + pivot_visits['android'], color='#31adc1', label='IOS')
plt.legend(loc='upper right')
plt.grid(linewidth=0.5, alpha=0.5)
plt.xlabel('Дата')
plt.xlim([conv_ads['date_group'].min(), conv_ads['date_group'].max()])
plt.xticks(x[::3], x[::3], rotation=45, fontsize=10)
plt.ylabel('Количество посещений сайта')
plt.title('Количество посещений сайта в отчётном периоде c разбивокй по платформам')
plt.tight_layout()
plt.savefig('./charts/visits_per_platform.png')

3. Итоговые регистрации

In [ ]:
x = conv_ads['date_group']
y = conv_ads['registrations']
plt.figure(figsize=(20, 5))
bars = plt.bar(x, y, color='#ab4bb5', label='Итоговые визиты')
for bar in bars:
    height = bar.get_height()
    plt.text(bar.get_x() + bar.get_width() / 2, height, f'{height:.0f}', ha='center', va='bottom')
#plt.legend()
plt.grid(axis='y', linewidth=0.5, alpha=0.5)
plt.xlabel('Дата')
plt.xlim([conv_ads['date_group'].min() - pd.Timedelta(days=2), conv_ads['date_group'].max() + pd.Timedelta(days=2)])
plt.xticks(x[::3], x[::3], rotation=45, fontsize=10)
plt.ylabel('Суммарное количество регистраций')
plt.title('Количество регистраций новых клиентов в отчётном периоде')
plt.tight_layout()
plt.savefig('./charts/registrations.png')

4. Итоговые регистрации с разбивкой по платформе: web, android, ios

In [ ]:
grouped_registrations = grouped_registrations.reset_index()

In [ ]:
pivot_gr_reg = grouped_registrations.pivot(index='date_group', columns='platform', values='registrations').fillna(0)

In [ ]:
pivot_gr_reg = grouped_registrations.pivot(index='date_group', columns='platform', values='registrations').fillna(0)
x = pivot_gr_reg.index
plt.figure(figsize=(20, 10))
plt.bar(x, pivot_gr_reg['android'], color='#aae11e', label='Android')
plt.bar(x, pivot_gr_reg['web'], color='#acb78e', label='Web')
plt.bar(x, pivot_gr_reg['ios'], color='#31adc1', label='IOS')
plt.legend(loc='upper right')
plt.grid(linewidth=0.5, alpha=0.5)
plt.xlabel('Дата')
plt.xlim([conv_ads['date_group'].min(), conv_ads['date_group'].max()])
plt.xticks(x[::3], x[::3], rotation=45, fontsize=10)
plt.ylabel('Количество регистраций')
plt.title('Количество регистраций новых клиентов в отчётном периоде c разбивокй по платформам')
plt.tight_layout()
plt.savefig('./charts/registrations_per_platform.png')

5. Конверсия по каждой платформе

In [ ]:
pivot_conv = conversions.pivot(index='date_group', columns='platform', values='conversion').fillna(0)

In [ ]:
pivot_conv = conversions.pivot(index='date_group', columns='platform', values='conversion').fillna(0)
x = pivot_conv.index
android_conv = pivot_conv['android']
web_conv = pivot_conv['web']
ios_conv = pivot_conv['ios']
plt.figure(figsize=(20, 15), constrained_layout=True)

plt.subplot(3, 1, 1)
plot = plt.plot(x, android_conv, color='#aae11e', label='Android', marker='o')
for date, value in android_conv.items():
    plt.text(date, value, f'{value:.0f}%')
plt.grid(axis='both', linewidth=0.5, alpha=0.5)
plt.xlabel('Дата', fontsize = 12)
plt.xlim(x.min() - pd.Timedelta(days=1), x.max() + pd.Timedelta(days=3))
plt.xticks(x[::5], x[::5], rotation=45, fontsize=10)
plt.ylabel('Конверсия, %', fontsize = 12)
plt.title('Конверсия Android', fontsize = 14)

plt.subplot(3, 1, 2)
plot = plt.plot(x, web_conv, color='#acb78e', label='Web', marker='o')
for date, value in web_conv.items():
    plt.text(date, value, f'{value:.0f}%')
plt.grid(axis='both', linewidth=0.5, alpha=0.5)
plt.xlabel('Дата', fontsize = 12)
plt.xlim(x.min() - pd.Timedelta(days=1), x.max() + pd.Timedelta(days=3))
plt.xticks(x[::5], x[::5], rotation=45, fontsize=10)
plt.ylabel('Конверсия, %', fontsize = 12)
plt.title('Конверсия Web', fontsize = 14)

plt.subplot(3, 1, 3)
plot = plt.plot(x, ios_conv, color='#31adc1', label='Итоговые визиты', marker='o')
for date, value in ios_conv.items():
    plt.text(date, value, f'{value:.0f}%')
plt.grid(axis='both', linewidth=0.5, alpha=0.5)
plt.xlabel('Дата', fontsize = 12)
plt.xlim(x.min() - pd.Timedelta(days=1), x.max() + pd.Timedelta(days=3))
plt.xticks(x[::5], x[::5], rotation=45, fontsize=10)
plt.ylabel('Конверсия, %', fontsize = 12)
plt.title('Конверсия Ios', fontsize = 14)

plt.savefig('./charts/conversion_per_platform.png')

6. Средняя конверсия

In [ ]:
conv_grouped = conv_ads[['date_group', 'visits', 'registrations']]
conv_grouped = conv_grouped.copy()
conv_grouped.loc[:, 'conversion_overall'] = conv_grouped['registrations'] / conv_grouped['visits'] * 100

In [ ]:
x = conv_grouped['date_group']
y = conv_grouped['conversion_overall']
plt.figure(figsize=(20, 5))
plot = plt.plot(x, y, color='#e66a56', label='Общая конверсия', marker='o')
for dot, dot_val in enumerate(y):
    plt.text(x[dot], y[dot] + 0.5, f'{dot_val:.0f}%')
plt.legend()
plt.grid(axis='y', linewidth=0.5, alpha=0.5)
plt.xlabel('Дата')
plt.xlim([conv_ads['date_group'].min(), conv_ads['date_group'].max()])
plt.xticks(x[::7], x[::7], rotation=45, fontsize=10)
plt.ylabel('Конверсия, %')
plt.title('Общая конверсия в отчётном периоде')
plt.tight_layout()
plt.savefig('./charts/conversion_overall.png')

7. Стоимости реклам

In [ ]:
x = conv_ads['date_group']
y = conv_ads['cost']
plt.figure(figsize=(20, 5))
plot = plt.plot(x, y, color='#cd4e00', label='Общая стоимость рекламы', marker='o')
for dot, dot_val in enumerate(y):
    plt.text(x[dot], y[dot] + 0.5, f'{dot_val:.0f} руб.')
#plt.legend()
plt.grid(axis='y', linewidth=0.5, alpha=0.5)
plt.xlabel('Дата')
plt.xlim([conv_ads['date_group'].min() - pd.Timedelta(days=1), conv_ads['date_group'].max() + pd.Timedelta(days=3)])
plt.xticks(x[::7], x[::7], rotation=45, fontsize=10)
plt.ylabel('Стоимость, руб.')
plt.title('Общая стоимость рекламы')
plt.tight_layout()
plt.savefig('./charts/campaign_cost_overall.png')

8. Визиты за весь период с цветовым выделением рекламной кампании

In [ ]:
campaigns_period = conv_ads[conv_ads['utm_campaign'] != 'none']
campaign_date = campaigns_period.groupby('utm_campaign')['date_group'].agg(['min', 'max']).reset_index()
#campaign_date['color'] = ['#8dbf40', '#cd5b45', '#32a4ba', '#f3ea16','#ababab']
campaign_date.columns = ['campaign', 'begin', 'end']

In [ ]:
colors = ['#8dbf40', '#cd5b45', '#32a4ba', '#f3ea16', '#ababab']
color_list = []
for campaign_number in range(len(campaign_date)):
    color_list.append(colors[campaign_number % 5])

In [ ]:
x = conv_ads['date_group']
y = conv_ads['visits']
plt.figure(figsize=(20, 7))
bars = plt.plot(x, y, color='#e66a56', label='Посещения', marker='o')
plt.grid(axis='both', linewidth=0.5, alpha=0.5)
plt.xlabel('Дата')
plt.xlim([conv_ads['date_group'].min() - pd.Timedelta(days=1), conv_ads['date_group'].max() + pd.Timedelta(days=1)])
plt.xticks(x[::7], x[::7], rotation=45, fontsize=10)
plt.ylabel('Суммарное количество посещений сайта')
plt.title('Количество посещений сайта')
plt.axhline(y.mean(), color='gray', linestyle='--', label='Среднее значение', alpha=0.5)
for campaign_number in range(len(campaign_date)):
    plt.axvspan(campaign_date['begin'].iloc[campaign_number], campaign_date['end'].iloc[campaign_number], color=color_list[campaign_number], alpha=0.3, label=campaign_date['campaign'].iloc[campaign_number])
plt.legend(loc='upper right')
plt.tight_layout()
plt.savefig('./charts/visits_with_campaigns.png')

9. Регистрации за весь период с цветовым выделением рекламной кампании

In [ ]:
x = conv_ads['date_group']
y = conv_ads['registrations']
plt.figure(figsize=(20, 7))
bars = plt.plot(x, y, color='#ab4bb5', label='Регистрации', marker='o')
plt.grid(axis='y', linewidth=0.5, alpha=0.5)
plt.xlabel('Дата')
plt.xlim([conv_ads['date_group'].min(), conv_ads['date_group'].max()])
plt.xticks(x[::7], x[::7], rotation=45, fontsize=10)
plt.ylabel('Суммарное количество регистраций')
plt.title('Количество регистраций новых клиентов')
plt.axhline(y.mean(), color='red', linestyle='--', label='Среднее значение', alpha=0.5)
for campaign_number in range(len(campaign_date)):
    plt.axvspan(campaign_date['begin'].iloc[campaign_number], campaign_date['end'].iloc[campaign_number], color=color_list[campaign_number], alpha=0.3, label=campaign_date['campaign'].iloc[campaign_number])
plt.legend(loc='upper right')
plt.tight_layout()
plt.savefig('./charts/registrations_with_campaigns.png')